In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import ast
import os
from fastparquet import ParquetFile
import time
from time import sleep
import librosa
from vagalume import lyrics
import difflib as di

## Evaluation of the organization data
The analysis is done for  evaluate the distribution of songs by fold and by time range.
 the entire dataset and for the subset of songs evaluated manually.

In [ ]:
data_GI_FO_AF_AA_L=pd.read_csv('lyrics_preprocessing_spotify_data.csv')

### Tempo bins distribution

In [ ]:
plt.figure()
sns.set()

data_1000 =data_GI_FO_AF_AA_L.copy()
data_1000 = data_GI_FO_AF_AA_L.dropna(subset=['manual_filt'])
ax = sns.histplot(data_GI_FO_AF_AA_L.tempo_bins, 
                  bins = 14,color = "darkorange")

ax = sns.histplot(data_1000.tempo_bins, 
                  bins = 14,color = "purple")
plt.axhline(y=100, color='k', linestyle=':',linewidth=3)
ax.figure.set_size_inches(16, 6)
ax.set_ylabel('Frequência', fontsize=14)
ax.set_xlabel('BPM', fontsize=14)


### Folds distribution

In [ ]:
plt.figure()
sns.set()

data_1000 =data_GI_FO_AF_AA_L.copy()
data_1000 = data_GI_FO_AF_AA_L.dropna(subset=['manual_filt'])
ax = sns.histplot(data_GI_FO_AF_AA_L.folds, 
                  bins = 20,color = "darkorange")

ax = sns.histplot(data_1000.folds, 
                  bins = 20,color = "purple")
plt.axhline(y=50, color='k', linestyle=':',linewidth=3)
ax.figure.set_size_inches(16, 6)
ax.set_ylabel('Frequência', fontsize=14)
ax.set_xlabel('BPM', fontsize=14)

### Tempo bins and folds distribution

In [ ]:
plt.figure()

ax = sns.histplot(data_GI_FO_AF_AA_L, x="tempo_bins", hue='folds',multiple="dodge",palette= "Oranges")
ax = sns.histplot(data_1000, x="tempo_bins", hue='folds',multiple="dodge",palette= "Purples")
ax.figure.set_size_inches(16, 6)
plt.axhline(y=5, color='b', linestyle=':',linewidth=3)
ax.set_ylabel('Count', fontsize=14)
ax.set_xlabel('Tempo bins', fontsize=14)

## Evaluation of analysis data
Problem found! Analysis data should be in list format, not string.

In [ ]:
data_GI_FO_AF_AA_L.columns

In [ ]:
data_GI_FO_AF_AA_L.bars[0]

In [ ]:
data_GI_FO_AF_AA_L['bars'] = data_GI_FO_AF_AA_L['bars'].apply(ast.literal_eval)
data_GI_FO_AF_AA_L['beats'] = data_GI_FO_AF_AA_L['beats'].apply(ast.literal_eval)
data_GI_FO_AF_AA_L['tatums'] = data_GI_FO_AF_AA_L['tatums'].apply(ast.literal_eval)

In [ ]:
data_GI_FO_AF_AA_L.bars[0]

### Problem solved 
But this problem occurs with the .csv format. So we will save the file in .parq too.

In [ ]:
data_GI_FO_AF_AA_L.to_parquet('forroset_update1.parq',index = False)

### Simplifying access to analysis data.

In [ ]:
def apeend_data(data,dado):
    starts= []
    duration = []
    confidence = []
    for j in range(len(dado)):
        starts.append(dado[j]['start'])
        duration.append(dado[j]['duration'])
        confidence.append(dado[j]['confidence'])
    data.append(starts)
    data.append(duration)
    data.append(confidence)
    return data
    
music_ana = []
for i in range(len(data_GI_FO_AF_AA_L['beats'])):
    data = []
    
    beats = data_GI_FO_AF_AA_L['beats'][i]
    bars = data_GI_FO_AF_AA_L['bars'][i]
    tatums =data_GI_FO_AF_AA_L['tatums'][i]
    data.append(data_GI_FO_AF_AA_L['track_id'][i])
    data = apeend_data(data,beats)
    data = apeend_data(data,bars)
    data = apeend_data(data,tatums)
    
    music_ana.append(data)

In [ ]:
columns= ['track_id','beats_start','beats_duration','beats_confidence','bars_start','bars_duration','bars_confidence','tatums_start','tatums_duration','tatums_confidence']
data_analises = pd.DataFrame(music_ana,columns =columns)

In [ ]:
data_GI_FO_AF_AA_L=data_GI_FO_AF_AA_L.drop(columns=["beats","bars","tatums"])

In [ ]:
data_GI_FO_AF_AA_L = data_GI_FO_AF_AA_L.merge(data_analises , how='left',on= "track_id")

In [ ]:
data_GI_FO_AF_AA_L.to_parquet('forroset_update1.parq',index = False)
data_GI_FO_AF_AA_L.to_csv('forroset_update1.csv',index = False)

### Inserting librosa beats anotation data

#### Downloading the audio files

In [ ]:
from mp3 import getMusics

tempo_inicial = time.time()
getMusics('forroset_update1.csv', 'files')
tempo_final = time.time()
duracao = tempo_final - tempo_inicial
print(duracao/3600)

In [ ]:
forroset = ParquetFile('forroset_update1.parq')
forroset = forroset.to_pandas()

musics = os.listdir("files")

#### Librosa beats data

In [ ]:
## Performing beats detection
beat_frames_obt = []
for music in musics:
    filename = ("/files/{}").format(music)
    y, sr = librosa.load(filename)
    onset_env = librosa.onset.onset_strength(y, sr=sr,aggregate=np.median)
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr, onset_envelope= onset_env, start_bpm = forroset.loc[forroset['track_id'] == music.split(".")[0]].tempo.tolist()[0])
    beat_frames_obt.append([beat_frames, music.split(".")[0]])
## Turning beat frames obtained into time
for i in range(len(beat_frames_obt)):
    beat_frames_obt[i][0] = librosa.frames_to_time(beat_frames_obt[i][0], sr=sr)

In [ ]:
## Adjusting the decimal digits of the beats obtained by Librosa
for i in range(len(beat_frames_obt)):
    for j in range(len(beat_frames_obt[i][0])):
        beat_frames_obt[i][0][j] = round(beat_frames_obt[i][0][j], 5)

In [ ]:
## Creating a DataFrame containing the list of detected beats and the respective identifier for each song
updates = pd.DataFrame(data = beat_frames_obt, columns = ["librosa_beats_start", "track_id"])
## Adding Updates to the forroset
updated_forroset = forroset.merge(updates, how='left',on='track_id')

In [ ]:
## Calculating the discrepancy measure of the beats obtained by Librosa
discrepancies = []
averages_spotify = []
## Listing the average distances of the beats obtained by the spotify API
for beats_list in updated_forroset['beats_start']:
    mean = sum(beatsDistances(beats_list))/len(beatsDistances(beats_list))
    averages_spotify.append(mean)
## Listing the average distances of the beats obtained by Librosa
averages_librosa = []
for beats_list in updated_forroset["librosa_beats_start"]:
    beats_dist = beatsDistances(beats_list)
    if len(beats_dist) != 0:
        mean = sum(beats_dist)/len(beats_dist)
        averages_librosa.append(mean)
    else:
        averages_librosa.append(None)
## Calculating discrepancy measure
for i in range(len(averages_spotify)):
    if averages_librosa[i] == None:
        discrepancies.append(None)
    else:
        discrepancy = abs((averages_librosa[i] - averages_spotify[i]))/averages_spotify[i]
        ## Adjusting the decimal digits of the discrepancy indices
        discrepancies.append(round(discrepancy,5))
## Adding a column of discrepancy indices

updated_forroset['librosa_discrepancy'] = discrepancies

In [ ]:
beats_list = []
for i in range(0,len(updated_forroset)):
    if updated_forroset['librosa_discrepancy'][i] == None:
        beats_list.append(None)
    else:
        beats_list.append(forroset_up['librosa_beats_start'][i])
        
updated_forroset['librosa_beats_start'] = beats_list

In [ ]:
updated_forroset.to_parquet('forroset_update1.parq',index = False)
updated_forroset.to_csv('forroset_update1.csv',index = False)

## Evaluation of lyrics data
Problem found! Lyrics data should be in list format, not string.

In [ ]:
data_GI_FO_AF_AA_L = ParquetFile('forroset_update1.parq')
data_GI_FO_AF_AA_L = data_GI_FO_AF_AA_L.to_pandas()

In [ ]:
data_GI_FO_AF_AA_L.lyrics[1]

In [ ]:
lyrics =  data_GI_FO_AF_AA_L[["track_id",'lyrics']].dropna()
lyrics['lyrics'] = lyrics['lyrics'].apply(ast.literal_eval)
data_GI_FO_AF_AA_L= data_GI_FO_AF_AA_L.drop(['lyrics','lyrics_confidence'], axis=1)

In [ ]:
forroset = data_GI_FO_AF_AA_L.merge(lyrics, how='left',on= "track_id")

### Problem solved 
Same problem that occurred with the analysis data.

In [ ]:
print(forroset.lyrics[1][0])
print()
print(forroset.lyrics[1][1])
print()
print(forroset.lyrics[1][2])

### FAIR's compilance problems
According to Vagalume API requirements it is necessary to insert the link of the lyrics.

In [ ]:
lyrics_data = forroset[['lyrics']].values 

letra = []
list_error = []
similarity = []
url = []
k=0
j=0
for i in range(0,len(lyrics_data)): 
    
    time.sleep(2)
    if i%50 ==0:
        print("Waiting a little bit.")
        print("Lyrics searched: ", i)
        print("Downloaded lyrics: ", k)
        time.sleep(5)
    
    if lyrics_data[i][0] != None:
        artist_name = lyrics_data[i][0][1]
        song_name = lyrics_data[i][0][0]

        try:
            result = lyrics.find(artist_name, song_name)

            if result.is_not_found():
                letra.append(None)
                similarity.append(None)
            else:
                s_artist = di.SequenceMatcher(None, artist_name.upper(), result.artist.name.upper()).ratio()
                s_name = di.SequenceMatcher(None, song_name.upper(), result.song.name.upper()).ratio()
                similarity.append({'name_similarity':s_name, 'artist_similarity':s_artist})
                 
                if s_name<1 or s_artist<1:
                    print(s_name, s_artist,song_name.upper(), result.song.name.upper())
                letra.append([result.song.name, result.artist.name,result.song.lyric,result.song.url])
                url.append(result.song.url)
                k+=1

        except:
            letra.append(None)
            similarity.append(None)
            url.append(None)
            time.sleep(5)
    else: 
        letra.append(None)
        similarity.append(None)
        url.append(None)

In [ ]:
forroset['lyrics']=letra

## Final forroset version

In [ ]:
forroset =forroset[['track_id', 'track', 'artist', 'artist_id', 'popularity', 'album',
       'album_id', 'track_year', 'duration_ms', 'uri', 'preview_url', 
                                
        'energy','liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness',
       'time_signature', 'danceability', 'key', 'loudness', 'valence', 'mode',
        
       'beats_start', 'beats_duration', 'beats_confidence', 'bars_start',
       'bars_duration', 'bars_confidence', 'tatums_start', 'tatums_duration',
       'tatums_confidence','librosa_beats_start','librosa_discrepancy', 
                                
       'tempo_bins', 'tempo_bins_max', 'genre_filt', 'folds','manual_filt', 
        
        'lyrics' ]]

In [ ]:
forroset.to_parquet('forroset.parq',index = False)
forroset.to_csv('forroset.csv',index = False)